# Trying to make an animation of Lagrangian particles

## Part of a project to quantify transports across the Greenland-Scotland section by source and destination

- Lagrangian particle tracking in VIKING20X model

- animation with matplotlib

## Technical preamble

In [ ]:
%matplotlib inline

In [ ]:


from pathlib import Path
import numpy as np
import xarray as xr
from datetime import datetime, timedelta
import seaborn as sns
import cmocean as co
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

np.warnings.filterwarnings('ignore')


## Parameters

In [ ]:
# Project path
project_path = Path.cwd() / '..' / '..' 
project_path = project_path.resolve()

# project_path = Path("C:/Users/sa05af/OneDrive - SAMS/Projects/parcels_quantitative_202004/")

# Parcels track data file
path_data_tracks = Path('data/processed/tracks/IcelandFaroe/') 
filename = 'Parcels_IFFForwards_1m_June2016_2000.nc'
filenameb = 'Parcels_IFFBackwards_1m_June2016_2000.nc'

# model mask file
data_path = Path("data/external/iAtlantic/")
experiment_name = "VIKING20X.L46-KKG36107B"
mesh_mask_file = project_path / data_path / "mask" / experiment_name / "1_mesh_mask.nc"


## Load data

### Forward tracks then backward tracks

In [ ]:
ds = xr.open_dataset(project_path / path_data_tracks / filename)
dsb = xr.open_dataset(project_path / path_data_tracks / filenameb)
ds_full = xr.concat([dsb,ds],dim='obs')


### Load model mesh mask

In [ ]:
mesh_mask = xr.open_dataset(mesh_mask_file)
mesh_mask = mesh_mask.squeeze()
mesh_mask = mesh_mask.set_coords(["nav_lon", "nav_lat", "nav_lev"])

bathy = mesh_mask.mbathy.rename("number of water filled points")

depth = (mesh_mask.e3t_0 * mesh_mask.tmask).sum("z")
# display(mesh_mask)

## Calculate some basic parameters from data for later

In [ ]:
nParticles = ds.sizes['traj']
nobsb = dsb.sizes['obs']


### Animate trajectories

In [ ]:
fig,ax = plt.subplots(figsize = (8,6))

# plot depth contours and land boundary from model bathymetry background

depth.isel(y=slice(1700, 2499), x=slice(1300, 2404)).plot.contour(
    x="nav_lon", y="nav_lat", colors = 'grey', levels = [200,800,1500,2000,2500,3500],zorder = 1
);
depth.isel(y=slice(1700, 2499), x=slice(1300, 2404)).plot.contour(
    x="nav_lon", y="nav_lat", colors = 'k', levels = [1]
);
plt.ylim(45,70)
plt.xlim(-30,10)

scat = dsb.isel(obs=-1).plot.scatter('lon','lat',hue='z',ax=ax,s=10,zorder=10)
title = ax.set_title(dsb.isel(obs=-1).time.data[0])


def animate(i):
    if i < nobsb:
        scat.set_offsets(np.c_[dsb.isel(obs=nobsb-1-i).lon.data,dsb.isel(obs=nobsb-1-i).lat.data])
        scat.set_array(dsb.isel(obs=nobsb-1-i).z.data)
        title = ax.set_title(dsb.isel(obs=nobsb-1-i).time.data[0])
    else:
        scat.set_offsets(np.c_[ds.isel(obs=i-nobsb).lon.data,ds.isel(obs=i-nobsb).lat.data])
        scat.set_array(ds.isel(obs=i-nobsb).z.data)
        title = ax.set_title(ds.isel(obs=i-nobsb).time.data[0])
  
    return scat,title

anim = FuncAnimation(
    fig, animate,frames = ds_full.sizes['obs'],interval = 100,blit = True)

In [ ]:
# <insert animation setup code here>
# to animate on screen (also contained in downloaded html file)
HTML(anim.to_html5_video())

# to save to a file
#anim.save('test.mp4')